<a href="https://colab.research.google.com/github/scsanjay/ml_from_scratch/blob/main/10.%20Density-Based%20Spatial%20Clustering%20of%20Applications%20with%20Noise%20(DBSCAN)/DBSCAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation of DBSCAN

In [8]:
import numpy as np

In [9]:
class DBSCAN():
  """
  DBSCAN (Density-Based Spatial Clustering of Applications with Noise) Implementation

  Parameters
  ----------
  eps : float, default=0.5
      The radius to consider.

  min_samples : int, default=5
      Minimum number of samples for a dense region.
  
  metric : {'minkowski', 'manhattan', 'euclidean'}, default = minkowski
      Distance measure to be used.

  p : int, default=2
      It is parameter for the minkowski distance (lp distance). It is used only
      when metric=minkowski.

  Attributes
  ----------
  labels_ : array of size n_samples
      Cluster labels of each data point. -1 if it's a noisy point.
  """

  def __init__ (self, eps=.5, min_samples=5, metric='minkowski', p=2):
    ''' initialize params '''
    self.eps = eps
    self.min_samples = min_samples
    self.metric = metric
    self.p = p

  def _getDistances(self, x_i, X):
    ''' get distance b/w a point and all the points'''
    if self.metric == 'euclidean':
      p = 2
    elif self.metric == 'manhattan':
      p = 1
    else:
      p = self.p
    # implement the distance calculation
    distances = np.power(np.sum(np.power(np.abs(x_i-X), p), axis=1), 1/p)
    return distances
  
  def _getCoreNeighbors(self, i, data, core_points, core_neighbors_all):
    ''' recursively get density connected points '''
    # actual core neighbors
    core_points = set(core_points)
    # neighbors of point i
    neighbor_points = set(data[i][0])
    # core neighbors of a point i
    core_neighbors = core_points.intersection(neighbor_points)
    # core neighbors that are not checked recursivly
    core_neighbors_not_traversed = core_neighbors.difference(core_neighbors_all)
    # density connected points
    core_neighbors_all = core_neighbors_all.union(core_neighbors)
    for j in core_neighbors_not_traversed:
      core_neighbors_all.union(self._getCoreNeighbors(j, data, core_points, core_neighbors_all))
    # return all density connected points
    return core_neighbors_all

    
  def fit(self, X):
    """
    It will find the clusters.

    Parameters
    ----------
    X : array of shape (n_samples, n_features)

    Returns
    -------
    self : object
    """

    data = {}
    core_points = []
    part_of_core_points = []
    # iterate over each data
    for i, x_i in enumerate(X):
      # calculate distances between x_i and all the data
      distances = self._getDistances(x_i, X)
      # get all indices where distance is less than eps
      distance_indices = np.argwhere(distances <= self.eps).flatten()
      point_label = 'na'
      # based on min_samples declare if a core point
      if len(distance_indices) >= self.min_samples:
        point_label = 'core'
        # core points list
        core_points.append(i)
        # all neighbors of all core point
        part_of_core_points.extend(distance_indices)
      # save neighbors of a point
      data[i] = [distance_indices, point_label]
    # for non-core points check if border point or noise point
    border_points = set()
    for key, value in data.items():
      if key not in core_points:
        if key in part_of_core_points:
          point_label = 'border'
          border_points.add(key)
        else:
          point_label = 'noise'
        data[key][1] = point_label
    # core points that are added to a cluster
    clustered_core_points = set()
    # list of clusters 
    clusters = []
    # iterate over each core point
    for i in core_points:
      # process if already not added to a cluster
      if i not in clustered_core_points:
        # get all density connected core points
        core_neighbors = self._getCoreNeighbors(i, data, core_points, set())
        # get all the border points of density connected core points
        # Note some implementation may process border points separately based 
        # on closest core point's cluster but sklearn does this way
        border_neighbors = set()
        for j in core_neighbors:
          border_neighbors.update(border_points.intersection(set(data[j][0])))
        # update core points that are added to a cluster
        clustered_core_points.update(core_neighbors)
        # merge core points and the border points
        core_neighbors.update(border_neighbors)
        # save the merged points as a cluster
        clusters.append(core_neighbors)
    # generate cluster labels based on clusters
    # noisy point will have -1
    self.labels_ = np.full(len(X), -1)
    for i, cluster in enumerate(clusters):
      self.labels_[list(cluster)] = i
    self.labels_ = list(self.labels_)
    # return self
    return self

  def fit_predict (self, X):
    """
    It will find the clusters and return labels.

    Parameters
    ----------
    X : array of shape (n_samples, n_features)

    Returns
    -------
    labels : array of shape (n_samples,)
        Cluster labels of each data point. -1 if it's a noisy point.
    """
    # fit the data and return labels
    return self.fit(X).labels_

# Validate the implementation

In [10]:
X = np.array([[1, 2], [2, 2], [2, 3],
              [8, 7], [8, 8], [7, 8], 
              [25, 80], 
              [-1, 0]])

## Custom implementation output

In [11]:
clustering = DBSCAN(eps=3, min_samples=3).fit(X)

In [12]:
clustering.labels_

[0, 0, 0, 1, 1, 1, -1, 0]

In [13]:
clustering.fit_predict(X)

[0, 0, 0, 1, 1, 1, -1, 0]

## sklearn's implementation output

In [14]:
from sklearn import cluster
clustering = cluster.DBSCAN(eps=3, min_samples=2).fit(X)
print(clustering.labels_)
print(clustering.fit_predict(X))

[ 0  0  0  1  1  1 -1  0]
[ 0  0  0  1  1  1 -1  0]


**We are getting same outputs. That means the custom implementation is correct.**